In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, LineString
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import Image
import selenium.webdriver
import shapely.geometry
import osmnx as ox
import pyproj
from geojson.feature import Feature, FeatureCollection
from shapely.geometry import Point, Polygon, LineString
phantom_path = '/Users/rkucharski/anaconda3/lib/node_modules/phantomjs-prebuilt/lib/phantom/bin/phantomjs'
import h3
import json

In [3]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
import seaborn as sns
import h3
import pandas as pd
import matplotlib.pyplot as plt
from dotmap import DotMap
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, collect_results  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate, simulate_parallel
from MaaSSim.traveller import travellerEvent
from MaaSSim.driver import driverEvent
from MaaSSim.utils import generate_demand

In [5]:
params = get_config('../../data/config/amsterdam.json')  # load configuration

In [6]:
params.paths.dumps = 'hexes'
params.times.patience = 1200
params.simTime = 4
params.parallel.nThread = 4
params.parallel.nReplications = 1
params.nV = 5
params.nP = 16

In [7]:
inData = load_G(inData, params)  # load network graph 

In [8]:
space = DotMap()
space.nP = [10]
space.nV = [1]
dict(space)

{'nP': [10], 'nV': [1]}

In [9]:
#simulate_parallel(params = params, search_space = space)
simulate(params = params)
res = collect_results('hexes')

16
0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
Name: pax_id, dtype: int64
11517
10935
30-05-23 00:38:27-WARNING-Setting up 4h simulation at 2023-05-29 22:40:20 for 5 vehicles and 16 passengers in Amsterdam, Netherlands
30-05-23 00:38:29-INFO--------------------	Starting simulation	-------------------


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
AMS = pd.read_csv("D://TIL6022//maassim//MaaSSim//MaaSSim//data//Amsterdam.csv",nrows = 5)
node = pd.DataFrame(AMS.columns[1:])
node.columns = ['node']
import numpy as np
def plot_hexagon(inData,params,node,APERTURE_SIZE = 8):
    for i in range(0,len(node['node'])):
        node['node'].iloc[i] = int(node['node'].iloc[i])
        if node['node'].iloc[i] not in inData.nodes.index:
            node['node'].iloc[i] = 0
    
    node = pd.DataFrame([i for i in node['node'] if i != 0])
    node.columns = ['node']
   
    node['x'] = node.apply(lambda row: inData.nodes.loc[row.node].x, axis = 1)
    node['y'] = node.apply(lambda row: inData.nodes.loc[row.node].y, axis = 1)
    node['hex_o_{}'.format(APERTURE_SIZE)] = node.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 

    ref = pd.concat([node['hex_o_{}'.format(APERTURE_SIZE)],node['x']],axis = 1)
    ref = pd.concat([ref,node['y']],axis = 1)
    ref = pd.concat([ref,node['node']],axis = 1)
    ref.columns = ['hash','x','y','node']

    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(node[col_geom].unique())+list(node[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)

    list_features = []
    for i, row in hexes.iterrows(): 
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)
    feature_collection = FeatureCollection(list_features)
    
    geojson_hexes = json.dumps(feature_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    base_map = folium.Map(location=CENTER, zoom_start=12,tiles = tile,zoom_control=False)

    folium.GeoJson(feature_collection, style_function=lambda x: {'fillColor': 'yellow',
                                                             'color': 'black',
                                                             'weight': 1,
                                                             'fillOpacity': 0.1,
                                                             'opacity':0.1}).add_to(base_map)
    return base_map,ref

base_map, ref = plot_hexagon(inData,params,node)
base_map

In [ ]:
ref['next_step'] = ref.apply(lambda row: h3.k_ring(row.hash,1),axis = 1)
ref['num_of_choices'] = ref.apply(lambda row: len(row.next_step),axis = 1)
ref = ref.set_index('node')
ref

,hash,x,y,next_step,num_of_choices
node,,,,,
6316199,881969c9b7fffff,4.888396,52.370173,"{881969c9b5fffff, 881969534bfffff, 881969c9b3f...",7
46379627,881969c9b7fffff,4.887774,52.370216,"{881969c9b5fffff, 881969534bfffff, 881969c9b3f...",7
46380013,881969c9b7fffff,4.887137,52.370258,"{881969c9b5fffff, 881969534bfffff, 881969c9b3f...",7
46380094,881969c9b7fffff,4.886584,52.370287,"{881969c9b5fffff, 881969534bfffff, 881969c9b3f...",7
46372441,881969c9b7fffff,4.887560,52.368749,"{881969c9b5fffff, 881969534bfffff, 881969c9b3f...",7
...,...,...,...,...,...
433807091,8819682447fffff,4.735187,52.419002,"{8819682445fffff, 881968246bfffff, 8819682447f...",7
46571771,8819682447fffff,4.736352,52.422490,"{8819682445fffff, 881968246bfffff, 8819682447f...",7
46581921,8819682445fffff,4.739251,52.426611,"{8819682445fffff, 8819682469fffff, 8819682713f...",7


In [ ]:
def random_walk(ref,num = 20):
    import random
    start = random.choice(list(set(ref.index)))
    location = ref.loc[start,'hash']
    sequence = []
    sequence.append(location)
    for time_winds in range(0,num):
        search = ref[ref['hash']==location].iloc[0]
        choice = search['next_step']  #possible locations for next step
        location = random.choice(list(choice)) #random walk
        sequence.append(location)
    return sequence

def trajectory(res,ref):
    traces = res.rides[['veh','pos','t']]
    for i in range(0,len(traces['pos'])):
        if traces['pos'].iloc[i] not in ref.index:
            traces['pos'].iloc[i] = (ref.index.to_series() - traces['pos'].iloc[i]).abs().idxmin()
    traces['hash'] = traces.apply(lambda row:ref.loc[row.pos,'hash'],axis = 1)
    traces = traces[traces['veh']==1]
    arr = np.array(traces['hash'])
    unique_values, index = np.unique(arr, return_index=True)
    sequence = arr[np.sort(index)]
    return sequence

def diff_events(res,ref):
    a = res.rides[res.rides['veh']==1]
    for i in range(0,len(a['pos'])):
        if a['pos'].iloc[i] not in ref.index:
            a['pos'].iloc[i] = (ref.index.to_series() - a['pos'].iloc[i]).abs().idxmin()
    print(a.event)
    a['hash'] = a.apply(lambda row:ref.loc[row.pos,'hash'],axis = 1)
    a = a[(a['event']=='RECEIVES_REQUEST') | (a['event']=='ARRIVES_AT_DROPOFF')]
    diffs = a['t'].diff()
    diffs.columns = ['wait']
    a = pd.concat([a[['pos','t','event','hash']], diffs], axis = 1)
    a.columns = ['pos','t','event','hash','wait']
    
    a['wait1'] = a['wait']
    for i in range(0,len(a.wait)-1):
        a['wait1'].iloc[i] = a['wait'].iloc[i+1]
    a['wait1'].iloc[len(a.wait)-1] = 0
    
    print(a[(a['wait']>30)&(a['event']=='RECEIVES_REQUEST')])
    print(a[(a['wait1']>30)&(a['event']=='ARRIVES_AT_DROPOFF')])
    seq1 = np.array(a[(a['wait']>30)&(a['event']=='RECEIVES_REQUEST')]['hash'])
    seq2 = np.array(a[(a['wait1']>30)&(a['event']=='ARRIVES_AT_DROPOFF')]['hash'])
    return seq1,seq2
    
    

def add_trajectory(sequence,color='blue'):
    hexes = pd.DataFrame(sequence)
    col_geom = 'hex_o_{}'.format(8)
    hexes.columns = [col_geom]
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)

    list_features = []
    for i, row in hexes.iterrows(): 
        feature = Feature(geometry = row["geom"],
                            id = row[col_geom],
                            properties = {"resolution": 9})
        list_features.append(feature)
    feature_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feature_collection)

    folium.GeoJson(feature_collection, style_function=lambda x: {'fillColor': color,
                                                             'color': color,
                                                             'weight': 1,
                                                             'fillOpacity': 0.1,
                                                             'opacity':0.1}).add_to(base_map)
    return base_map

#sequence = trajectory(res,ref)
seq1, seq2 = diff_events(res,ref)
base_map = add_trajectory(seq1,'green')
base_map = add_trajectory(seq2,'red')

base_map



0                    STARTS_DAY
1                     OPENS_APP
2              RECEIVES_REQUEST
3               ACCEPTS_REQUEST
4      IS_ACCEPTED_BY_TRAVELLER
                ...            
82            ARRIVES_AT_PICKUP
83    MEETS_TRAVELLER_AT_PICKUP
84          DEPARTS_FROM_PICKUP
85           ARRIVES_AT_DROPOFF
86                   ENDS_SHIFT
Name: event, Length: 313, dtype: object
             pos      t             event             hash    wait   wait1
9   2.001333e+09   3167  RECEIVES_REQUEST  8819695301fffff  1508.0   994.0
37  4.635900e+07  10194  RECEIVES_REQUEST  8819695347fffff  2872.0  1157.0
44  4.636286e+07  12832  RECEIVES_REQUEST  881969535dfffff  1481.0  1000.0
51  4.629056e+07  13935  RECEIVES_REQUEST  8819695357fffff   103.0   890.0
30  1.595260e+09   5136  RECEIVES_REQUEST  8819695247fffff   428.0  1522.0
44  4.637697e+07  10516  RECEIVES_REQUEST  881969c9b7fffff  2778.0  2693.0
9   3.412535e+09    986  RECEIVES_REQUEST  8819695359fffff   121.0   551.0
16  4.64